## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление
1. [Шаг 1. Откройте файл с данными и изучите общую информацию](#step1)  
    1.1 [Вывод к Шагу 1](#conclusion1)
    
        
2. [Шаг 2. Предобработка данных](#step2)  
    2.1   [Обработка пропусков](#step2.1)  
    2.2   [Анализ уникальных значений](#step2.2)  
    2.3   [*Вывод к обработке пропусков и анализу уникальных значений*](#conclusion2.3)  
    2.4   [Замена типов данных](#step2.4)  
    2.5   [*Вывод к замене типов данных*](#conclusion2.5)  
    2.6   [Обработка дубликатов](#step2.6)  
    2.7   [*Вывод к обработке дубликатов*](#conclusion2.7)  
    2.8   [Лемматизация](#step2.8)  
    2.9   [*Вывод к лемматизации*](#conclusion2.9)  
    2.10  [Категоризация данных](#step2.10)  
    2.11  [*Вывод к категоризации*](#conclusion2.11)  
    
    
3. [Шаг 3. Ответы на вопрос](#step3)  
    3.1   [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#question1)  
    3.2   [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#question2)  
    3.3   [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#question3)  
    3.4   [Как разные цели кредита влияют на его возврат в срок?](#question4)    
    
    
4. [Шаг 4. Общий вывод](#step4)

## Шаг 1. Откройте файл с данными и изучите общую информацию.<a id="step1"></a>   
</div>


In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info() # общая информация о данных
print(data)
data.columns # проверяем названия столбцов






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее 

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')


### 1.1 Вывод к шагу 1 <a id="conclusion1"></a>   
</div>


В предоставленных данных есть пропуски (столбцы: 'days_employed' и 'total_income'). 
В столбце "education" есть одинаковые по смыслу, но разные по написанию значения, к примеру (Среднее и среднее).
В столбце 'days_employed' есть артефакты (очень большие значения) и отрицательные значения, которые в дальнейшем приведем к абсолютным..
В целом, данные подлежат дальнейшей обработке.


## Шаг 2. Предобработка данных<a id="step2"></a>   
</div>

### 2.1 Обработка пропусков<a id="step2.1"></a>   
</div>

Пропуски предположительно есть в столбцах 'days_employed' и 'total_income', также возможны нулевые, некорректные значения в 'dob_years'

In [2]:
print(data.loc[data['dob_years'] == 0]) # проверим нет ли нулевого возраста


       children  days_employed  dob_years education  education_id  \
99            0  346541.618895          0   Среднее             1   
149           0   -2664.273168          0   среднее             1   
270           3   -1872.663186          0   среднее             1   
578           0  397856.565013          0   среднее             1   
1040          0   -1158.029561          0    высшее             0   
...         ...            ...        ...       ...           ...   
19829         0            NaN          0   среднее             1   
20462         0  338734.868540          0   среднее             1   
20577         0  331741.271455          0   среднее             1   
21179         2    -108.967042          0    высшее             0   
21313         0   -1268.487728          0   среднее             1   

               family_status  family_status_id gender income_type  debt  \
99           женат / замужем                 0      F   пенсионер     0   
149                в 

101 человек скрывает свой возраст. Это явная ошибка, поэтому для каждого типа 'income_type' посчитаем медианный возраст и подставим его для каждого типа в таблицу

In [3]:
dob_years_median = data.groupby('income_type')['dob_years'].median() 
print(dob_years_median)



income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64


In [4]:
for items in dob_years_median.iteritems():
    income_type = items[0]
    median = items[1]
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == income_type), 'dob_years'] = median
    
                                          
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'безработный'), 'dob_years'] = dob_years_median[0]
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'в декрете'), 'dob_years'] = dob_years_median[1]
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'госслужащий'), 'dob_years'] = dob_years_median[2]
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'компаньон'), 'dob_years'] = dob_years_median[3]
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'пенсионер'), 'dob_years'] = dob_years_median[4]
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'предприниматель'), 'dob_years'] = dob_years_median[5]
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'сотрудник'), 'dob_years'] = dob_years_median[6]
#data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'студент'), 'dob_years'] = dob_years_median[7]

#Проверим остался ли ещё кто-то скрытный
data.loc[data['dob_years'] == 0, 'dob_years'].value_counts()



Series([], Name: dob_years, dtype: int64)

Узнаем количество пропущенных значений

In [5]:
data.isnull().sum() # по 2174 пропуска в 'days_employed' и 'total_income'

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Просто обнулить 2174 из 21524 значений нежелательно, так как это составляет около 10% от общего количества, а значит существенно влияет на статистику. Поэтому заменим пропущенные значения на медианные. Рассмотрим медианные значения для каждого типа занятости 'income_type'.

In [6]:
data.loc[data['total_income'].isnull(), 'income_type'].value_counts()
total_income_median = data.groupby('income_type')['total_income'].median()
print(f'Медианные значения по типу')
print(round(total_income_median))


Медианные значения по типу
income_type
безработный        131340.0
в декрете           53829.0
госслужащий        150448.0
компаньон          172358.0
пенсионер          118514.0
предприниматель    499163.0
сотрудник          142594.0
студент             98202.0
Name: total_income, dtype: float64


In [7]:
for items in total_income_median.iteritems():
    income_type = items[0]
    median = items[1]
    data.loc[(data['total_income'].isnull()) & (data['income_type'] == income_type), 'total_income'] = median

#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'безработный'), 'total_income'] = total_income_median[0]
#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'в декрете'), 'total_income'] = total_income_median[1]
#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'госслужащий'), 'total_income'] = total_income_median[2]
#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'компаньон'), 'total_income'] = total_income_median[3]
#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'пенсионер'), 'total_income'] = total_income_median[4]
#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'предприниматель'), 'total_income'] = total_income_median[5]
#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'сотрудник'), 'total_income'] = total_income_median[6]
#data.loc[(data['total_income'].isnull()) & (data['income_type'] == 'студент'), 'total_income'] = total_income_median[7]

#Проверим остался ли ещё кто-то без дохода. 
print(data.loc[data['total_income'].isnull(), 'income_type'].value_counts())
#Округлим бесконечные копейки
data['total_income'] = round(data['total_income'])

Series([], Name: income_type, dtype: int64)


In [8]:
print('Всего людей по типам занятости')
print(data['income_type'].value_counts())
print()
print('Количество пропущенных значений')
print(data.loc[data['days_employed'].isnull(), 'income_type'].value_counts())



print(data.loc[data['days_employed'] > 50000, 'income_type'].value_counts())
# видим, что в основном слишком большой стаж у пенсионеров и двух безработных. Скорее всего, здесь речь идет
# об ошибки в форматировании или человеческий фактов во время заполнения ячеек (к примеру во время копирования)


   

# заменим на медианные значения пропущенные значения по столбцу 'days_employed'
days_employed_median = data.groupby('income_type')['days_employed'].median()


for items in days_employed_median.iteritems():
    income_type = items[0]
    median = items[1]
    data.loc[(data['days_employed'].isnull()) & (data['income_type'] == income_type), 'days_employed'] = median

    
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'безработный'), 'days_employed'] = days_employed_median[0]
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'в декрете'), 'days_employed'] = days_employed_median[1]
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'госслужащий'), 'days_employed'] = days_employed_median[2]
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'компаньон'), 'days_employed'] = days_employed_median[3]
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'пенсионер'), 'days_employed'] = days_employed_median[4]
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'предприниматель'), 'days_employed'] = days_employed_median[5]
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'сотрудник'), 'days_employed'] = days_employed_median[6]
#data.loc[(data['days_employed'].isnull()) & (data['income_type'] == 'студент'), 'days_employed'] = days_employed_median[7]



Всего людей по типам занятости
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Количество пропущенных значений
сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
пенсионер      3443
безработный       2
Name: income_type, dtype: int64


In [9]:
print('Cлишком молодые пенсионеры c большим стажем')
print(data.loc[(data['income_type'] == 'пенсионер') & (data['days_employed'] > 50000) & (data['dob_years'] < 30)])
print()


Cлишком молодые пенсионеры c большим стажем
       children  days_employed  dob_years education  education_id  \
1242          0  334764.259831       22.0   Среднее             1   
3619          0  365213.306266       24.0   среднее             1   
12507         0  379492.102505       27.0   среднее             1   
12753         1  329781.704997       27.0   среднее             1   
13953         0  376824.585817       27.0   среднее             1   
16166         0  364348.197352       26.0   среднее             1   
19417        -1  350340.760224       28.0   среднее             1   
19439         0  389397.167577       26.0    высшее             0   

               family_status  family_status_id gender income_type  debt  \
1242   Не женат / не замужем                 4      F   пенсионер     0   
3619         женат / замужем                 0      F   пенсионер     0   
12507  Не женат / не замужем                 4      F   пенсионер     0   
12753        женат / замужем      

### 2.2 Анализ уникальных значений<a id="step2.2"></a>   
</div>

In [10]:
# Посмотрим уникальные значения во всех категориальных столбцах, а также в столбце количества детей
print(data[['children', 'education', 'family_status','gender', 'income_type', 'debt']].apply(lambda x: x.value_counts()).T.stack())
# И правда классный способ, спасибо! Обнаружился ещё один маленький артефакт в виде XNA у gender

children       -1                          47.0
               0                        14149.0
               1                         4818.0
               2                         2055.0
               3                          330.0
               4                           41.0
               5                            9.0
               20                          76.0
education      ВЫСШЕЕ                     274.0
               Высшее                     268.0
               НАЧАЛЬНОЕ                   17.0
               НЕОКОНЧЕННОЕ ВЫСШЕЕ         29.0
               Начальное                   15.0
               Неоконченное высшее         47.0
               СРЕДНЕЕ                    772.0
               Среднее                    711.0
               УЧЕНАЯ СТЕПЕНЬ               1.0
               Ученая степень               1.0
               высшее                    4718.0
               начальное                  250.0
               неоконченное высшее      

In [11]:
print(data.loc[data['gender'] == 'XNA']) # посмотрим, кто такой этот XNA.
#раз он в гражданском браке, то заменим его XNA, на мужской пол. 
data['gender'] = data['gender'].replace('XNA', 'M')
print(data.loc[data['gender'] == 'XNA']) # проверим, все ли ок

       children  days_employed  dob_years            education  education_id  \
10701         0   -2358.600502       24.0  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

       total_income               purpose  
10701      203905.0  покупка недвижимости  
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [12]:
data['children'] = abs(data['children']) #исправляем отрицательные значения по детям
data['days_employed'] = abs(data['days_employed']) #исправляем отрицательные значения по стажу
print('У кого 20 детей')
print(data.loc[(data['children'] > 6)]) # обнаружили людей, у которых количество детей 20. Маловероятно, похоже на техническую ошибку и детей на самом деле 2. 
data.loc[(data['children'] > 6), 'children'] = 2 # исправляем количество детей
print('Проверяем исправление')
print(data.loc[(data['children'] > 6)]) # проверяем исправление


У кого 20 детей
       children  days_employed  dob_years education  education_id  \
606          20     880.221113       21.0   среднее             1   
720          20     855.595512       44.0   среднее             1   
1074         20    3310.411598       56.0   среднее             1   
2510         20    2714.161249       59.0    высшее             0   
2941         20    2161.591519       39.0   среднее             1   
...         ...            ...        ...       ...           ...   
21008        20    1240.257910       40.0   среднее             1   
21325        20     601.174883       37.0   среднее             1   
21390        20    1547.382223       53.0   среднее             1   
21404        20     494.788448       52.0   среднее             1   
21491        20     173.954460       27.0   среднее             1   

         family_status  family_status_id gender income_type  debt  \
606    женат / замужем                 0      M   компаньон     0   
720    женат / за

In [13]:
print(data.isnull().sum()) #проверяем ушли ли все пропуски
data

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253876.0,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145886.0,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267629.0,дополнительное образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.0,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43.0,среднее,1,гражданский брак,1,F,компаньон,0,224792.0,операции с жильем
21521,0,343937.404131,67.0,среднее,1,женат / замужем,0,F,пенсионер,0,156000.0,сделка с автомобилем
21522,1,2113.346888,38.0,среднее,1,гражданский брак,1,M,сотрудник,1,89673.0,недвижимость
21523,3,3112.481705,38.0,среднее,1,женат / замужем,0,M,сотрудник,1,244093.0,на покупку своего автомобиля


### 2.3 Вывод к обработке пропусков и анализу уникальных значений<a id="conclusion2"></a>   
</div>


В данных обнаружилось значительно больше проблем, чем казалось на первый взгляд.  
Пропущенные значения заменили на медианные (с разбивкой по типу), а отрицательные на абсолютные.  
Исправили техническую ошибку при заполнении по количеству детей. По 20 детей многовато всё-таки.  
Убрали неправильный пол.


Слишком большие значения в стаже пока игнорируем. Эта проблема касается пенсионеров и безработных.
Можно было бы попровать взять средний пенсионный возраст и вычесть от текущего возраста, но возникает ряд проблем.
1. Во-первых, от профессии к профессии возраст выхода на пенсию разный.
2. Во-вторых, в данных явно есть ошибка по возрасту некоторых "пенсионеров". Они либо другого типа (человеческая ошибка), либо существовала другая категория, которую в итоге объединили с "пенсионерами". 
    
В целом данные готовы к дальнейшей обработке.

    
    
    



### 2.4 Замена типа данных<a id="step2.4"></a>   
</div>

In [14]:
#Ещё раз посмотрим на данных и типы данных
data.dtypes





children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

Заменим тип даных у столбца 'dob_years' на целочисленный

In [15]:
data['dob_years'] = data['dob_years'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data['debt'] = data['debt'].astype('bool')

print('Типы данных после замены')
data.dtypes #проверим выполнение 

Типы данных после замены


children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                  bool
total_income         int64
purpose             object
dtype: object

### 2.5 Вывод к замене типов данных <a id="conclusion2.5"></a>   
</div>

Заменили тип данных у столбца возраст клиента в годах, так как никакой необходимости в том, чтобы иметь не целочисленное значение, нет.  
Применили метод astype, так как пропущенные значения убрали выше, а значит to_numeric здесь не нужен.  
Для 'days_employed' и 'total_income' изменили тип на int64, а 'debt' на 'bool'.  
*Всё ради экономии памяти*


### 2.6 Обработка дубликатов <a id="step2.6"></a>   
</div>

Принять решение о наличии или отсутствии строк дубликатов может только после обработки столбцов 'education' и 'purpose'. Так как при минимальном изучении, можно обратить внимание, что наличиствует ряд одинаковых по смыслу, но разных по написанию целей и уровней образования.

Начнём с того, что все текстовые блоки переведём в нижний регистр.

In [16]:
print(data['education'].value_counts()) #посмотрим какие есть варианты обучения сейчас
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

print('Проверим исправление')
print(data['education'].value_counts())
print('Жаль, что так мало ученых степеней')

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
Проверим исправление
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
Жаль, что так мало ученых степеней


На будущее надо предусмотреть, чтобы в обучении не было возможности писать "как хочется", а предлагать только варианты на выбор

In [17]:
print(data['purpose'].value_counts()) #посмотрим какие есть цели сейчас
print()
print(f"Общее количество целей: {len(data['purpose'].value_counts())}")

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Здесь видно 38 вариантов, которые просто так не объединишь. Значит будем использовать лемматизацию. Но предварительно можем посмотреть, нет ли полных дубликатов с такими разными целями.

In [18]:
print('Сортировка дубликатов по доходу')
print(data[data.duplicated()].sort_values('total_income', ascending = False).head(10))
print()
print('Сортировка дубликатов по возрасту')
print(data[data.duplicated()].sort_values('dob_years', ascending = False).head(10))
print()
print(f"Всего дубликатов {data.duplicated().sum()}")


data = data.drop_duplicates().reset_index(drop = True) # удаляем дубликаты, восстанавливем индексы.
data.duplicated().sum() # проверяем, что всё хорошо


Сортировка дубликатов по доходу
       children  days_employed  dob_years education  education_id  \
15991         0           1547         51   среднее             1   
13878         1           1547         31   среднее             1   
17774         1           1547         40   среднее             1   
19369         0           1547         45   среднее             1   
19387         0           1547         38    высшее             0   
17379         0           1547         54    высшее             0   
10697         0           1547         40   среднее             1   
14432         2           2689         36    высшее             0   
18349         1           2689         30    высшее             0   
16902         2           1574         39   среднее             1   

          family_status  family_status_id gender  income_type   debt  \
15991  гражданский брак                 1      f    компаньон  False   
13878   женат / замужем                 0      f    компаньон  F

0

In [19]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,False,253876,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,False,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,False,145886,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,False,267629,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,False,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,f,компаньон,False,224792,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,False,156000,сделка с автомобилем
21451,1,2113,38,среднее,1,гражданский брак,1,m,сотрудник,True,89673,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,m,сотрудник,True,244093,на покупку своего автомобиля


### 2.7 Вывод к обработке дубликатов <a id="conclusion2.7"></a>   
</div>


Удалили дубликаты (полностью идентичные строки), предварительно переведя всю текстовую часть массива данных в нижний регистр, это позволило убрать очевидные дубликаты. Возможно, после лемматизации столбца 'purpose' сможем найти ещё некоторое количество дубликатов. Для 'education' использовали метод value_counts, дубликаты удаляли классическим drop_duplicates.

### 2.8 Лемматизация<a id="step2.8"></a>   
</div>

In [20]:
#Для начала импортируем необходимые для лемматизации библиотеки.
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

data['purpose'].value_counts() # ещё раз посмотрим на виды целей

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Среди целей можно выделить несколько основных: свадьба, покупка автомобиля, покупка/строительство недвижимости и жилья (потенциально можно объединить), образование. Важно отметить тот факт, что некоторые покупают недвижимость не для себя, а для коммерческих целей, это тоже нужно предусмотреть. 

Таким образом список слов для лемматизации сформируем следующий: свадьба, недвижимость, коммерческий, жилье, сдача, автомобиль, образование. Где жилье и недвижимость отнесем к личным целям, а сдачу и коммерческий к "коммерции".



In [21]:
purposes = ['свадьба', 'коммерческий', 'сдача', 'недвижимость', 'жилье', 'автомобиль', 'образование']
#чтобы более большая группа "недвижимость" не съела "коммерческую" часть, ставим их в начало списка.
def lemmatization(purpose):
    lemma = m.lemmatize(purpose)
    for word in purposes:
        if word in lemma:
            lemma = word
    return lemma

print("Группируем цели по более общей цели и проверяем")
data['purpose_general'] = data['purpose'].apply(lemmatization)
data

Группируем цели по более общей цели и проверяем


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_general
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,False,253876,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,False,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,False,145886,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,False,267629,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,False,158616,сыграть свадьбу,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,f,компаньон,False,224792,операции с жильем,жилье
21450,0,343937,67,среднее,1,женат / замужем,0,f,пенсионер,False,156000,сделка с автомобилем,автомобиль
21451,1,2113,38,среднее,1,гражданский брак,1,m,сотрудник,True,89673,недвижимость,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,m,сотрудник,True,244093,на покупку своего автомобиля,автомобиль


In [22]:
data['purpose_general'].value_counts()

недвижимость    5040
автомобиль      4306
образование     4013
жилье           3809
свадьба         2324
коммерческий    1311
сдача            651
Name: purpose_general, dtype: int64

Как и планировали, объединим недвижимость с жильем, а сдачу с коммерческим целью.

In [23]:
data.loc[data['purpose_general'] == 'недвижимость', 'purpose_general'] = 'личная недвижимость'
data.loc[data['purpose_general'] == 'жилье', 'purpose_general'] = 'личная недвижимость'
data.loc[data['purpose_general'] == 'коммерческий', 'purpose_general'] = 'коммерция'
data.loc[data['purpose_general'] == 'сдача', 'purpose_general'] = 'коммерция'
print(data['purpose_general'].value_counts()) # все ли объединились
print()
print ('И даже никого не потеряли')
print (data.isnull().sum())


личная недвижимость    8849
автомобиль             4306
образование            4013
свадьба                2324
коммерция              1962
Name: purpose_general, dtype: int64

И даже никого не потеряли
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
purpose_general     0
dtype: int64


In [24]:
#проверим не появилось ли новых дубликатов
print(f'Есть ли новые дубликаты')
print(data[data.duplicated()])
print('Фуф, новых нет')

Есть ли новые дубликаты
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, purpose_general]
Index: []
Фуф, новых нет


### 2.9 Вывод к лемматизации<a id="conclusion2.9"></a>   
</div>


Лемматизировали цели. Выделили 5 основных: личная недвижимость, автомобиль, образование, свадьба и коммерция. Похоже, что наш банк специализируется на ипотеке. Надеюсь, у них хорошие ставки. 

### 2.10 Категоризация данных<a id="step2.10"></a>   
</div>

Выше мы уже создали категоризацию по цели кредита. Можем создать еще одну категорию, которая отражает зависимость между количеством детей и возвратом кредита в срок. Для этого напишем функцию children_debt

In [25]:
data['children'].value_counts() #больше 5 детей ни у кого нет, поэтому:

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [26]:
def children_debt(row):
    children = row['children']
    debt = row['debt']
    if children == 1 and debt == 0:
        return 'Есть 1 ребенок и платит в срок'
    if children == 2 and debt == 0:
        return 'Есть 2 ребенка и платит в срок'
    if children  >=3  and  debt == 0:
        return 'Есть 3 и более детей и платит в срок'
    if children == 1 and debt == 1:
        return 'Есть 1 ребенок и не платит в срок'
    if children == 2 and debt == 1:
        return 'Есть 2 ребенка и не платит в срок'
    if children  >=3  and  debt == 1:
        return 'Есть 3 и более детей и не платит в срок'    
    if children == 0 and debt == 0:
        return 'Нет детей и платит в срок'
    if children > 0 and debt == 1:
        return 'Есть дети и не платит в срок'
    if children == 0 and debt == 1:
        return 'Нет детей и не платит в срок'
#row_values = [3,0] #проверяем работоспособность функции
#row_columns = ['children','debt']
#row = pd.Series(data = row_values, index = row_columns)
#print(children_debt(row))

#Создаём дополнительный столбец 'children_return'
data['children_return'] = data.apply(children_debt, axis = 1)
#print(data)



### 2.11 Вывод к категоризации данных <a id="conclusion2.11"></a>   
</div>


Категоризовали данные по целям и зависимостью между наличием детей и платежами в срок. Это поможет ответить на вопросы по проекту

### Шаг 3. Ответы на вопросы <a id="step3"></a>   
</div>

### - Есть ли зависимость между наличием детей и возвратом кредита в срок? <a id="question1"></a>   
</div>

Посмотрим на количество в каждой категории

In [27]:
print(data['children_return'].value_counts())


Нет детей и платит в срок                  13028
Есть 1 ребенок и платит в срок              4410
Есть 2 ребенка и платит в срок              1926
Нет детей и не платит в срок                1063
Есть 1 ребенок и не платит в срок            445
Есть 3 и более детей и платит в срок         349
Есть 2 ребенка и не платит в срок            202
Есть 3 и более детей и не платит в срок       31
Name: children_return, dtype: int64


In [28]:
nochild_pay = data.loc[data['children_return'] == 'Нет детей и платит в срок']['children_return'].count()
nochild_nopay = data.loc[data['children_return'] == 'Нет детей и не платит в срок']['children_return'].count()
child1_pay = data.loc[data['children_return'] == 'Есть 1 ребенок и платит в срок']['children_return'].count()
child1_nopay = data.loc[data['children_return'] == 'Есть 1 ребенок и не платит в срок']['children_return'].count()
child2_pay = data.loc[data['children_return'] == 'Есть 2 ребенка и платит в срок']['children_return'].count()
child2_nopay = data.loc[data['children_return'] == 'Есть 2 ребенка и не платит в срок']['children_return'].count()
child3_pay = data.loc[data['children_return'] == 'Есть 3 и более детей и платит в срок']['children_return'].count()
child3_nopay = data.loc[data['children_return'] == 'Есть 3 и более детей и не платит в срок']['children_return'].count()
procent_nochild = nochild_nopay / (nochild_nopay + nochild_pay) 
procent_child1 = child1_nopay / (child1_pay + child1_nopay)
procent_child2 = child2_nopay / (child2_pay + child2_nopay)
procent_child3 = child3_nopay / (child3_pay + child3_nopay)

print('Процент неплательщиков у кого нет детей: {:.2%}'.format(procent_nochild))
print('Процент неплательщиков с 1 ребенком: {:.2%}'.format(procent_child1))
print('Процент неплательщиков с 2 детьми: {:.2%}'.format(procent_child2))
print('Процент неплательщиков с 3 и более детьми: {:.2%}'.format(procent_child3))


Процент неплательщиков у кого нет детей: 7.54%
Процент неплательщиков с 1 ребенком: 9.17%
Процент неплательщиков с 2 детьми: 9.49%
Процент неплательщиков с 3 и более детьми: 8.16%


### Вывод

Зависимость между наличием детей и возвратом кредита в срок есть. И она не в пользу тех, у кого дети есть. Это можно объяснить тем, что дети всегда в приоритете (а значит и траты на них), поэтому и возникает большее количество просрочек.
При этом чаще всего допускают просрочку родители двух детей. Чуть меньше нарушений кредитного договора у семей с одним ребёнком.  
Интересно, что если детей трое и более, то процент просрочек уменьшается. В любом случае, до тех, у кого детей нет, они всё равно немного недотягивают. 

### - Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id="question2"></a>   
</div>

Используем сводную таблицу, сгрупирированную по семейному положению, столбцами будет наличие задолженности, значениями - подсчёт по 'gender'. И добавляем столбец 'ratio', который даст нам соотношение по просрочке.

In [29]:
data_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)

data_pivot.sort_values('ratio', ascending = False)


debt,False,True,ratio
family_status,,,
не женат / не замужем,2536,274,0.098
гражданский брак,3763,388,0.093
женат / замужем,11408,931,0.075
в разводе,1110,85,0.071
вдовец / вдова,896,63,0.066


### Вывод

Зависимость между семейным положением и возвратом кредита в срок есть. 
Как видно из таблицы, если человек находится в браке (или был в нём), то он более ответственно относится к возврату кредита в срок.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?<a id="question3"></a>   
</div>

Для ответа на этот вопрос, нам нужно написать функцию, которая будет определять уровень дохода. Но для ей работы нам нужно определить уровни дохода.

In [30]:

print ('Минимальное доход:', data['total_income'].min()) 
print ('Медианный доход:', data['total_income'].median())
print ('Средний доход:', data['total_income'].mean())
print ('Максимальный доход:',data['total_income'].max())
print (data['total_income'].describe()) 
def income_level(income):
    if income < 107624:
        return 'низкий'
    if 107624 <= income <= 142594:
        return 'ниже среднего'
    if 142594 < income <= 195821:
        return 'выше среднего'
    else:
        return 'высокий'
# income_group(150000) - проверяем работоспособность

Минимальное доход: 20667
Медианный доход: 142594.0
Средний доход: 165320.05243777385
Максимальный доход: 2265604
count    2.145400e+04
mean     1.653201e+05
std      9.818731e+04
min      2.066700e+04
25%      1.076240e+05
50%      1.425940e+05
75%      1.958212e+05
max      2.265604e+06
Name: total_income, dtype: float64


Используя функцию income_level добавим столбец уровень дохода к таблице

In [31]:
data['income_level'] = data['total_income'].apply(income_level) #добавляем столбец
data.head() # проверяем данные

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_general,children_return,income_level
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,False,253876,покупка жилья,личная недвижимость,Есть 1 ребенок и платит в срок,высокий
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,False,112080,приобретение автомобиля,автомобиль,Есть 1 ребенок и платит в срок,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,False,145886,покупка жилья,личная недвижимость,Нет детей и платит в срок,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,False,267629,дополнительное образование,образование,Есть 3 и более детей и платит в срок,высокий
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,False,158616,сыграть свадьбу,свадьба,Нет детей и платит в срок,выше среднего


Снова строим сводную таблицу, группируем по уровню дохода, столбцы - наличие задолженности, значения - подсчёт по 'gender'. Плюс после формирования таблицы добавим соотношение по просрочке

In [32]:
data_pivot = data.pivot_table(index = ['income_level'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('income_level', ascending = False)

debt,False,True,ratio
income_level,,,
низкий,4937,427,0.080
ниже среднего,4996,483,0.088
выше среднего,4799,448,0.085
высокий,4981,383,0.071


### Вывод
Определенная зависимость между уровнем дохода и возвратом кредита выявлена. Не столь строгая, как в случае с семейным положением, но:
Разделив все данные на квартиле с помощью метода describe, мы получили четыре условных уровня дохода 'income_level'.
Да, однозначно можно сказать, что люди с высоким доходом допускают меньше просрочек, около 7,1%.
При этом ближе всего к ним за тем люди с низким доходм и их 8%.
Самые опасные как раз средние категории, они допускают от 8,5% (доход выше среднего) до 8,8% просрочек (доход ниже среднего)



### - Как разные цели кредита влияют на его возврат в срок?<a id="question4"></a>   
</div>

Построим сводную таблицу. Группируем по цели кредита, столбцы - наличие задолженности, значения - подсчёт по 'gender'. Плюс после формирования таблицы добавим соотношение по просрочке

In [33]:
data_pivot = data.pivot_table(index = ['purpose_general'], columns = 'debt', values = 'gender', aggfunc = 'count')
data_pivot['ratio'] = round(data_pivot[1] / (data_pivot[0] + data_pivot[1]), 3)
data_pivot.sort_values('ratio', ascending = False)

debt,False,True,ratio
purpose_general,,,
автомобиль,3903,403,0.094
образование,3643,370,0.092
свадьба,2138,186,0.080
коммерция,1811,151,0.077
личная недвижимость,8218,631,0.071


### Вывод


Наиболее ответственными клиентами банка являются те, кто берет кредит на покупку недвижимости (как личной, так и коммерческой). За ними идут молодожены (что коррелируется с выводом по зависимости от семейного положения). 
Хуже всего дела обстоят с кредитами на образование и автомобили. 

Можно предположить, что банк внимательнее относятся к заемщикам на большие суммы (покупка недвижимости), собирается больший пакет документов. 

Засчет меньшей суммы кредита на образование, больше клиентов способны обеспечить (по документам) оплату. Тоже справедливо и для автокредитов, кроме этого, менеджеры автосалонов всеми правдами и неправдами стараются обеспечить одобрение клиента в банке. После покупки автомобиля, если с транспортом что-то случается (авария, кража), то происходит "моральное выгорание" клиента, он не готов платить деньги за пустоту. На этот случай банки обязывают приобретать страховку, но за время урегулирования бюрократических вопросов со страховой компанией, многие клиенты допускают просрочки.






## Шаг 4. Общий вывод <a id="step4"></a>   
</div>  

1. Клиенты с детьми допускают просрочку на 1,7 п.п. чаще, чем без детей.  
2. Замужние/женатые допускают просрочку на 2,5 п.п реже, чем незамужние/неженатые.  
3. Люди с высоким уровнем дохода на 1,7 п.п. реже допускают просрочку, чем с доходом ниже среднего.  
4. Клиенты, берущие кредит на недвижимость допускают просрочку на 2,3 п.п. реже, чем идущие в банк за кредитом на автомобиль.

Таким образом, с точки зрения банка, наибольшее внимание стоит уделять семейному положению клиента и цели кредита. 

Наличие детей скорее увеличивает возможность просрочки.
Основываться только на уровне дохода также недостаточно, так как строго распределения в зависимости нет.


*Идеальный для банка клиент: замужний/женатый клиент без детей с высоким уровнем дохода, покупающий недвижимость.*